_Since we announced [our collaboration with the World Bank and more partners to create the Open Traffic platform](https://mapzen.com/blog/announcing-open-traffic/), we’ve been busy. We’ve shared [two](https://mapzen.com/blog/open-traffic-osmlr-technical-preview/) [technical](https://mapzen.com/blog/osmlr-2nd-technical-preview/) previews of the OSMLR linear referencing system. Now we’re ready to share more about how we’re using [Mapzen Map Matching](https://mapzen.com/blog/map-matching/) to “snap” GPS-derived locations to OSMLR segments, and how we’re using a data-driven approach to evaluate and improve the algorithms._

# A "data-driven" approach to improving map-matching, Part II

In the [last](https://mapzen.com/blog/map-matching-validation/) blog post on Mapzen's map-matching service, we showed you how Mapzen uses synthetic GPS data to validate the matches that our map-matching algorithm generates. In this post, we'll dive a _bit_ deeper into the internals of the algorithm itself to see how we can fine-tune the map-matching parameters based on the expected fidelity of the data we receive. 

## 0. Setup test environment

In [1]:
from matplotlib import pyplot as plt
from matplotlib import cm, colors, patheffects
import numpy as np
import os
import glob
import urllib
import json
import pandas as pd
from random import shuffle, choice
import pickle
import sys; sys.path.insert(0, os.path.abspath('..'));
import validator.validator as val
%matplotlib inline

## 1. Map-matching Parameters

The Open Traffic Reporter map-matching service is based on the Hidden Markov Model (HMM) design of [Newton and Krumm (2009)](http://research.microsoft.com/en-us/um/people/jckrumm/Publications%202009/map%20matching%20ACM%20GIS%20camera%20ready.pdf). HMM's define a class of models that utilize a directed graph structure to represent the probability of observing a particular sequence of events -- or in our case, a particular sequence of road segments that defines a route. If you'd like a more detailed explanation of how HMM's work in the context of map-matching, please see the excellent primer [here](https://github.com/valhalla/meili/blob/master/docs/meili/algorithms.md). For our purposes, it is enough to know that  HMM's can be, in general, parameterized by two probability distributions - the _emission_ probability and the _transition_ probability - which are governed by parameters $\sigma_z$ and $\beta$, respectively.

## $\sigma_z$ 

The parameter $\sigma_z$ appears in the **emission probability** equation below which expresses the likelihood of recording a GPS measurement $z_t$ from road segment $r_i$ as a Gaussian probability density function (PDF), where $x_{t,i}$ is the point on road segment $r_i$ nearest the measurement $z_t$ at time $t$.

### $$ p(z_t|r_i) = \frac{1}{\sqrt{2 \pi \sigma_z}} e^{-0.5(\frac{||z_t - x_{t,i}||_{\text{great circle}}}{\sigma_z})^2}$$

In practice, $\sigma_z$ can be thought of as an estimate of the standard deviation of the GPS noise. The more we trust our measurements, the lower our $\sigma_z$ should be. Newton and Krumm (2009) derive $\sigma_z$ from the median absolute deviation over their dataset, arriving at a value of 4.07 m. 

### $\beta$

The $\beta$ parameter comes from the equation for our **transition probability**, which is written as follows:




### $$p(d_t) = \frac{1}{\beta}e^{\frac{-d_t}{\beta}}$$

where $d_t$ is the difference between the great circle distance and route-traveled distance between time $t$ and $t+1$. We use this exponential PDF for our **transition probability** because Newton and Krumm (2009) found that for two successive GPS measurements, the differences in length between the "great circle" distance (i.e. "as-the-crow-flies" distances) and the corresponding route-traveled distance are exponentially distributed.

<img src="newson_and_krumm_exp.png" alt="Drawing" width="50%" align="center"/>
<center><i>From Newson and Krumm (2009)</i></center>

In this context, $\beta$ can be thought of as the expected difference between great circle distances and route distance traveled. We would expect a higher $\beta$ value for more circuitous, less direct routes.

## 2. Grid search to find optimal parameter values

Although we can empirically derive our parameter values using the equations above, we can also just search the space of feasible parameter values and store the ones that give us the optimal match scores. This is a common machine learning approach for algorithm optimization, typically referred to as a **grid search** or parameter sweep. Here, we perform a grid search for each of our specified sample rates so that we can store the optimal parameter values for each one.

#### a) Define search space

Because this grid search takes place in such high dimensional space - $\text{noise levels} \times \text{sample rates} \times \beta \ \text{values} \times \sigma\  \text{values} \times \text{# routes}$ - it is recommended to use a smaller number of discrete noise levels for the section of the analysis (< 10). Rather than estimating match scores in increments of 5 m of noise, we use increments of 20 below

In [2]:
noiseLevels = np.linspace(0, 100, 6)   # in meters
sampleRates = [1, 5, 10, 20, 30]   # in seconds
betas = np.arange(0.5, 10.5, 0.5)   # no units
sigmaZs = np.arange(0.5, 10.5, 0.5)    #no units

#### b) Search and score

The function `grid_search_hmm_params(`) will iterate through the sample rates, noise levels,  $\beta$  values and  $\sigma_z$  values for each of the sythetic routes that have already been generated. Because this process can be so slow, the results are saved incrementally in the background, and will be recombined in a later step.

In [3]:
val.grid_search_hmm_params(cityName, routeList, sampleRates, noiseLevels, betas, sigmaZs)

In the last post we saw this chart which nicely laid out the relationship between data quality and map-matching performance:

<img src="newson_krumm_score.png" alt="Drawing" width="70%" align="center"/>

Sample rate and measurement noise are the two defining characteristics of GPS data quality. GPS noise is clearly positively correlated with match error, but sample rate is a bit more complex. 